# Import Python module

In [49]:
import matplotlib.pyplot as plt
import time

In [50]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

In [51]:
import numpy as np 
import cv2
from pywt import dwt,idwt
import os

#### 임시
***

In [52]:
phone = "galaxyS7"
p = "phone1"
img_num = 1

***

### 1. 자유 Crop 여부 확인 및 복원

현재는 편의를 위해 화면 사이즈를 미리 알고 복원하지만, 실제 상황에서는 모든 화면 크기에 대해 복원한 후 각 이미지마다 워터마크 추출을 시도한다.

In [53]:
def dewatermark(x):
    """Removes all the significant bits and place the least significant bits to the most significant for watermark extraction."""
    x = format(x, '#010b')
    first = '0b'
    mid = x[-2:]
    third = '000000'
    y = first + mid + third
    result = int(y, 2)
    return np.uint8(result)

In [54]:
screen_size = {"galaxyS7":(1920,1080,3), "iPhoneX":(2436,1125,3),
               "LGv40":(3120,1440,3)}

In [55]:
emd_url = 'embed/captured_{}_img{}_crop.png'.format(p,img_num)

embed_img = cv2.imread(emd_url)

if embed_img.shape != screen_size[phone]:
    # 복원
    ex_b = embed_img[:,:,0]
    vecdewatermark = np.vectorize(dewatermark)
    extract = vecdewatermark(ex_b)

    col = 0
    row = 0
    c_dict = {}

    for i in range(0, extract.shape[1]):
        c_dict[i] = 0
        tmp = extract[:,i][0]

        for k in range(1,extract.shape[0]):
            if tmp == extract[:,i][k]:
                c_dict[i] +=1
            tmp = extract[:,i][k]

    col = [key for key, value in c_dict.items() if value > extract.shape[0]-500]

    r_dict = {}

    if len(col) != 0:
        for i in range(0,extract.shape[0]):
            r_dict[i] = 0
            tmp = extract[i][0]
            for k in range(1, extract.shape[1]):
                if tmp == extract[i][k]:
                    r_dict[i] +=1
                tmp = extract[i][k]

        row = [key for key, value in r_dict.items() if value > extract.shape[1]-500]

    if len(col) ==0 or len(row) == 0:
        print("추출 불가능")

    else:
        # 가운데가 추출되는 경우
        ## 복원 후 Extract
#         mid = (extract.shape[0]//2, extract.shape[1]//2)

        point= [0,0]
        
        img_shape = screen_size[phone][:2]
        
        row.sort()
        col.sort()

        # 모두 4개가 다 있는 경우
        if len(col) == 4 and len(row) == 4:
            point[0] = (img_shape[0] -4)//2 - row[0]
            point[1] = (img_shape[1] -4)//2 - col[0]
            
            insert_point = (point[1],point[0])
        
            background = (0,0,0)
            pad_size = (img_shape[1],img_shape[0]) #캡쳐 사이즈

            image = Image.new('RGB', pad_size, background)
            im = Image.open(emd_url)
            image.paste(im ,insert_point)
            
            re_url = "embed/reshape/reshape_{}_img{}.png".format(p,img_num)
            image.save(re_url)
            embed_img = cv2.imread(re_url)
        else:
            pass



# 추출 안되면 불가능

embed_img = embed_img.astype(np.float32)

### 2. extract

제약1) wm_shape 알아야 함(14,140 통일)

In [56]:
def block_get_wm(block):
    
    mod = 32
    
    block_dct = cv2.dct(block)
    U,s,V = np.linalg.svd(block_dct)
    max_s = s[0]
    
    if max_s%mod > mod/2:
        wm_1 = 255
    else:
        wm_1= 0
    
    return wm_1

In [57]:
start = time.time()

# font10
wm_shape = (13,148)

dwt_deep = 1
block_shape = (8,8)

embed_img_YUV = cv2.cvtColor(embed_img, cv2.COLOR_BGR2YUV)


if not embed_img_YUV.shape[0]%(2**dwt_deep)==0:
    temp = (2**dwt_deep)-embed_img_YUV.shape[0]%(2**dwt_deep)
    embed_img_YUV = np.concatenate((embed_img_YUV,np.zeros((temp,embed_img_YUV.shape[1],3))),axis=0)
if not embed_img_YUV.shape[1]%(2**dwt_deep)==0:
    temp = (2**dwt_deep)-embed_img_YUV.shape[1]%(2**dwt_deep)
    embed_img_YUV = np.concatenate((embed_img_YUV,np.zeros((embed_img_YUV.shape[0],temp,3))),axis=1)

# assert embed_img_YUV.shape[0]%(2**dwt_deep)==0
# assert embed_img_YUV.shape[1]%(2**dwt_deep)==0

coeffs_Y = dwt(embed_img_YUV[:,:,0],'haar')
ha_Y = coeffs_Y[0]
coeffs_Y = coeffs_Y[1]

shape0_int,shape1_int = ha_Y.shape[0]//block_shape[0], ha_Y.shape[1]//block_shape[1]
block_index0,block_index1 = np.meshgrid(np.arange(shape0_int),np.arange(shape1_int))
b_flat_idx0, b_flat_idx1 = block_index0.flatten(),block_index1.flatten()
length = b_flat_idx0.size

ha_block_shape = (int(ha_Y.shape[0]/block_shape[0]),int(ha_Y.shape[1]/block_shape[1]),block_shape[0],block_shape[1])
strides = ha_Y.itemsize*(np.array([ha_Y.shape[1]*block_shape[0],block_shape[1],ha_Y.shape[1],1]))

ha_Y_block = np.lib.stride_tricks.as_strided(ha_Y.copy(),ha_block_shape,strides)
# coeffs_Y_block = np.lib.stride_tricks.as_strided(coeffs_Y.copy(),ha_block_shape,strides)

extract_wm = np.array([])


for i in range(length):
    
    wm = block_get_wm(ha_Y_block[b_flat_idx0[i],b_flat_idx1[i]])
    
#     wm2 = block_get_wm(coeffs_Y_block[b_flat_idx0[i],b_flat_idx1[i]])

    # i가 워터마크의 크기를 벗어난 경우
    if i<wm_shape[0]*wm_shape[1]:
        extract_wm  = np.append(extract_wm,wm)
        
#         extract_wm2 = np.append(extract_wm2,wm2)
        
    else:
        times = i//(wm_shape[0]*wm_shape[1])
        ii = i%(wm_shape[0]*wm_shape[1])
        extract_wm[ii] = (extract_wm[ii]*times + wm)/(times+1)
        
#         extract_wm2[ii] = (extract_wm2[ii]*times + wm2)/(times+1)

print("걸린시간: ",time.time()-start)

걸린시간:  1.1868247985839844


In [58]:
new = extract_wm.copy()

new[new < 128] = 0
new[new >= 128] = 255

out_wm_name = "extract/extract_{}_img{}_crop.png".format(p,img_num)
out_new_wm_name = "extract/extract_{}_img{}_crop_new.png".format(p,img_num)

In [59]:
cv2.imwrite(out_wm_name,extract_wm.reshape(wm_shape[0],wm_shape[1]))
cv2.imwrite(out_new_wm_name,new.reshape(wm_shape[0],wm_shape[1]))

True